In [29]:

import os, math
import pandas as pd
import numpy as np
import en_core_web_sm
nlp = en_core_web_sm.load()
from gensim.models import Word2Vec as word2vec
from os import path as ospath

In [30]:
root_path = ospath.dirname(os.getcwd())
print(root_path)
model = word2vec.load(ospath.join(root_path,'nlp project/Word2vec_model'))

C:\Users\Prithika


In [31]:
def most_similar(token, n):
    token = token.lower()
    tokens = [token]
    closest_values = [1]
    try:
        list_similar = model.most_similar(positive=[token],topn=n)
        
        for tupl in list_similar:
            tokens.append(tupl[0])
            closest_values.append(tupl[1])
    except:
        pass   
    return tokens, closest_values

In [32]:
resumes = pd.read_csv('resume.csv', sep='\t')
resumes = resumes.set_index('Unnamed: 0')
jd = pd.read_csv(os.getcwd()+'\jd.csv')
print(jd['description'])

0    "software engineer c, c++, java, uml, xamp, ag...
Name: description, dtype: object


In [33]:
def lemm(word):
    try:
        symb = '''~'`!@#$%^&*)(_+-=}{][|\:;",./<>?'''
        mod_token = ''
        
        for c in word:
            if (c not in symb):
                mod_token += c.lower()

        docx = nlp(mod_token)

        if (len(mod_token) == 0 or docx[0].is_stop):
            return None
        else:
            return docx[0].lemma_
    except:
        return None 

In [34]:
for job_desc in jd['description']:
    resume_description = ''
    for w in job_desc.split():
        lemm_w = lemm(w)
        if (lemm_w is not None):
            resume_description += (lemm_w + ' ')
    print(resume_description)
    break

software engineer c c java uml xamp agile defencecommunication client successful expand company develop air defence system information system target system communication system require additional software engineer support design evolution requirement software code identification test requirement development component test harness support design code review activity derisk design evolution qualification software engineer degree software engineering computer science math physics equivalent good degree pref 1st 1 meng equiv essential requirement software engineer experience follow object orient analysis design preferably uml unix linux windows c c java html php candidate defence background experience agile development advantage experience xamp technology useful desirable requirement software engineer experience follow realtime design mathematical algorithm code network protocol oo case tool structured design method datum voice network computer telephony integration cti appreciation follow

In [35]:
word_val = {}
no_of_sim_w = 2
for w_r in resume_description.split():
    closest_words, sim = most_similar(w_r, no_of_sim_w)
    for i in range(len(closest_words)):
        word_val[closest_words[i]] = word_val.get(closest_words[i], 0)+sim[i]
        print(closest_words[i], word_val[closest_words[i]])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


software 1
hardware 0.6482969522476196
tool 0.5211771130561829
engineer 1
engineering 0.7069295644760132
engineered 0.5901373624801636
c 1
b 0.7182981967926025
f 0.5554901957511902
c 2
b 1.436596393585205
f 1.1109803915023804
java 1
scala 0.6912347674369812
python 0.6561834812164307
uml 1
venn 0.6297123432159424
diagrams 0.5596497058868408
xamp 1
sqlmap 0.6282532811164856
mvpcdl 0.6203561425209045
agile 1
scrum 0.577690601348877
development 0.5234850645065308
defencecommunication 1
client 1
server 0.730476975440979
clients 0.594319224357605
successful 1
unsuccessful 0.6542235612869263
successfull 0.5245006084442139
expand 1
extend 0.6629175543785095
expanding 0.6154885292053223
company 1
corporation 0.7230262756347656
ceo 0.6287192106246948
develop 1
developing 0.652402937412262
build 0.6494753360748291
air 1
oxygen 0.5212373733520508
water 0.5150661468505859
defence 1
defense 0.7732242345809937
najdorf 0.5807723999023438
system 1
systems 0.641305685043335
mechanism 0.49700844287872314

In [36]:
num_resumes = 21
count = {}
idf = {}
for w in word_val.keys():
    count[w] = 0
    for i in range(num_resumes):
        try:
            if w in resumes.loc(0)['skill'][i].split() or word in resumes.loc(0)['exp'][i].split():
                count[w] += 1
        except:
            pass
    if (count[w] == 0):
        count[w] = 1
    idf[w] = math.log(num_resumes/count[w])
print(count)
print(idf)

{'software': 2, 'hardware': 1, 'tool': 4, 'engineer': 1, 'engineering': 1, 'engineered': 1, 'c': 1, 'b': 1, 'f': 1, 'java': 3, 'scala': 1, 'python': 1, 'uml': 1, 'venn': 1, 'diagrams': 1, 'xamp': 1, 'sqlmap': 1, 'mvpcdl': 1, 'agile': 1, 'scrum': 1, 'development': 3, 'defencecommunication': 1, 'client': 1, 'server': 3, 'clients': 1, 'successful': 1, 'unsuccessful': 1, 'successfull': 1, 'expand': 1, 'extend': 1, 'expanding': 1, 'company': 1, 'corporation': 1, 'ceo': 1, 'develop': 3, 'developing': 1, 'build': 1, 'air': 1, 'oxygen': 1, 'water': 1, 'defence': 1, 'defense': 1, 'najdorf': 1, 'system': 8, 'systems': 1, 'mechanism': 1, 'information': 1, 'info': 1, 'informations': 1, 'target': 1, 'destination': 1, 'targets': 1, 'communication': 5, 'communications': 1, 'comms': 1, 'require': 1, 'involve': 2, 'allow': 1, 'additional': 1, 'extra': 1, 'auxiliary': 1, 'support': 1, 'supporting': 1, 'supports': 1, 'design': 4, 'designer': 1, 'designers': 1, 'evolution': 1, 'emergence': 1, 'evolutionar

In [26]:
score = {}
for i in range(num_resumes):
    score[i] = 0
    try:
        for w in word_val.keys():
            tf = resumes.loc(0)['skill'][i].split().count(w) + resumes.loc(0)['exp'][i].split().count(w)
            score[i] += word_val[w]*tf*idf[w]
    except:
        pass

0 0.0 software 9 0.0
0 0.0 hardware 5.834672570228577 0.0
0 0.0 tool 5.690594017505646 0.0
0 0.0 engineer 6.706929564476013 0.0
0 0.0 engineering 5.241577386856079 0.0
0 0.0 engineered 3.5408241748809814 0.0
0 0.0 c 6 0.0
0 0.0 b 4.309789180755615 0.0
0 0.0 f 3.332941174507141 0.0
0 0.0 java 3 0.0
0 0.0 scala 2.0737043023109436 0.0
0 0.0 python 1.968550443649292 0.0
0 0.0 uml 3 0.0
0 0.0 venn 1.8891370296478271 0.0
0 0.0 diagrams 1.6789491176605225 0.0
0 0.0 xamp 3 0.0
0 0.0 sqlmap 1.8847598433494568 0.0
0 0.0 mvpcdl 1.8610684275627136 0.0
0 0.0 agile 3 0.0
0 0.0 scrum 1.7330718040466309 0.0
0 0.0 development 4.570455193519592 0.0
0 0.0 defencecommunication 1 0.0
0 0.0 client 1 0.0
0 0.0 server 0.730476975440979 0.0
0 0.0 clients 0.594319224357605 0.0
0 0.0 successful 1 0.0
0 0.0 unsuccessful 0.6542235612869263 0.0
0 0.0 successfull 0.5245006084442139 0.0
0 0.0 expand 1 0.0
0 0.0 extend 0.6629175543785095 0.0
0 0.0 expanding 0.6154885292053223 0.0
0 0.0 company 2 0.0
0 0.0 corporation 

1 16.9286501713745 design 6 0.0
1 16.9286501713745 designer 3.209580659866333 0.0
1 16.9286501713745 designers 3.2043936252593994 0.0
1 16.9286501713745 evolution 2 0.0
1 16.9286501713745 emergence 1.1699018478393555 0.0
1 16.9286501713745 evolutionary 1.0902670621871948 0.0
1 16.9286501713745 requirement 4 0.0
1 16.9286501713745 requirements 2.864898204803467 0.0
1 16.9286501713745 restriction 2.33852481842041 0.0
1 16.9286501713745 code 3 0.0
1 16.9286501713745 program 1.6863927841186523 0.0
1 16.9286501713745 script 1.5780061483383179 0.0
1 16.9286501713745 identification 1 0.0
1 16.9286501713745 identifying 0.45653873682022095 0.0
1 16.9286501713745 tagging 0.41807374358177185 0.0
1 16.9286501713745 test 2.68131422996521 0.0
1 16.9286501713745 tests 2.0942978858947754 0.0
1 16.9286501713745 testing 2.36262845993042 0.0
1 16.9286501713745 prototyping 1.7475793361663818 0.0
1 16.9286501713745 component 1 0.0
1 16.9286501713745 components 0.638621985912323 0.0
1 16.9286501713745 eleme

2 164.21241811501457 science 1 0.0
2 164.21241811501457 sciences 0.6304487586021423 0.0
2 164.21241811501457 psychology 0.6233943700790405 0.0
2 164.21241811501457 math 1 0.0
2 164.21241811501457 maths 0.8157658576965332 0.0
2 164.21241811501457 mathematics 1.350952684879303 0.0
2 164.21241811501457 physics 1 0.0
2 164.21241811501457 chemistry 0.7196189165115356 0.0
2 164.21241811501457 biology 0.6561659574508667 0.0
2 164.21241811501457 equivalent 1 0.0
2 164.21241811501457 analogous 0.6312097311019897 0.0
2 164.21241811501457 equal 0.5647801756858826 0.0
2 164.21241811501457 good 2 0.0
2 164.21241811501457 bad 1.346861481666565 0.0
2 164.21241811501457 decent 1.309349536895752 0.0
2 164.21241811501457 pref 1 0.0
2 164.21241811501457 prefs 0.7367343306541443 0.0
2 164.21241811501457 preferences 0.6489847898483276 0.0
2 164.21241811501457 1st 1 0.0
2 164.21241811501457 1 1 0.0
2 164.21241811501457 meng 1 0.0
2 164.21241811501457 hons 0.5674545764923096 0.0
2 164.21241811501457 aerospac

4 723.9381751865411 constructor 0.5358366966247559 0.0
4 733.0717424997114 orient 1 9.13356731317027
4 733.0717424997114 visualize 0.5453997850418091 0.0
4 733.0717424997114 visualise 0.5218580961227417 0.0
4 739.1607873751582 analysis 1 6.089044875446846
4 739.1607873751582 analyses 0.6936036348342896 0.0
4 739.1607873751582 modeling 0.501259446144104 0.0
4 739.1607873751582 preferably 1 0.0
4 739.1607873751582 ideally 0.5285884141921997 0.0
4 739.1607873751582 preferrably 0.35120654106140137 0.0
4 739.1607873751582 unix 2 0.0
4 739.1607873751582 nix 2.8733590841293335 0.0
4 739.1607873751582 posix 1.392472267150879 0.0
4 739.1607873751582 linux 2.6197802424430847 0.0
4 739.1607873751582 ubuntu 2.074284553527832 0.0
4 739.1607873751582 windows 1 0.0
4 740.2593996638263 html 1 1.0986122886681098
4 740.2593996638263 htm 0.591211199760437 0.0
4 740.2593996638263 xhtml 0.5737585425376892 0.0
4 740.2593996638263 php 1 0.0
4 740.2593996638263 lighttpd 0.540486752986908 0.0
4 740.25939966382

5 11.956791549672097 appreciation 1 0.0
5 11.956791549672097 gratitude 0.6316518187522888 0.0
5 11.956791549672097 admiration 0.5969173908233643 0.0
5 11.956791549672097 techniques 0.6728505492210388 0.0
5 11.956791549672097 strategy 1 0.0
5 11.956791549672097 strategies 0.6583006381988525 0.0
5 11.956791549672097 approach 0.6165152788162231 0.0
5 11.956791549672097 configuration 1 0.0
5 11.956791549672097 config 0.6647350788116455 0.0
5 11.956791549672097 settings 0.5765025019645691 0.0
5 11.956791549672097 management 1 0.0
5 11.956791549672097 managing 0.5913525819778442 0.0
5 13.657320264436239 organizational 0.5585535168647766 3.044522437723423
5 13.657320264436239 personal 1 0.0
5 14.510116789289834 professional 0.43825072050094604 1.9459101490553132
5 14.510116789289834 personnal 0.41573837399482727 0.0
5 14.510116789289834 attribute 1 0.0
5 14.510116789289834 attributes 0.6233428120613098 0.0
5 14.510116789289834 enum 0.558282196521759 0.0
5 14.510116789289834 flexibility 1 0.0


...

11 14.476213440653806 systems 3.206528425216675 0.0
11 14.476213440653806 mechanism 2.4850422143936157 0.0
11 14.476213440653806 information 1 0.0
11 14.476213440653806 info 0.7569488286972046 0.0
11 14.476213440653806 informations 0.7292569279670715 0.0
11 14.476213440653806 target 1 0.0
11 14.476213440653806 destination 0.5292235612869263 0.0
11 14.476213440653806 targets 0.49063628911972046 0.0
11 14.476213440653806 communication 3 0.0
11 14.476213440653806 communications 2.309588670730591 0.0
11 14.476213440653806 comms 1.7042083740234375 0.0
11 14.476213440653806 require 1 0.0
11 14.476213440653806 involve 0.7528709173202515 0.0
11 14.476213440653806 allow 0.6582288146018982 0.0
11 14.476213440653806 additional 1 0.0
11 14.476213440653806 extra 0.793278157711029 0.0
11 14.476213440653806 auxiliary 0.5114134550094604 0.0
11 14.476213440653806 support 2 0.0
11 14.476213440653806 supporting 1.2496987581253052 0.0
11 14.476213440653806 supports 1.0736067295074463 0.0
11 24.42558190027

14 261.65802824438185 organized 0.6448134183883667 0.0
14 261.65802824438185 organised 0.5806669592857361 0.0
14 265.776554532876 technique 1.7515393495559692 2.3513752571634776
14 265.776554532876 procedure 0.6536369323730469 0.0
14 265.776554532876 datum 1 0.0
14 265.776554532876 sublist 0.4286709427833557 0.0
14 265.776554532876 tuple 0.42479556798934937 0.0
14 265.776554532876 voice 1 0.0
14 265.776554532876 speech 0.587144672870636 0.0
14 265.776554532876 voices 0.548427939414978 0.0
14 265.776554532876 telephony 1 0.0
14 265.776554532876 ril 0.5620831251144409 0.0
14 265.776554532876 telecom 0.5053160190582275 0.0
14 268.12792979003945 integration 1 2.3513752571634776
14 268.12792979003945 integrating 0.51705002784729 0.0
14 268.12792979003945 integrate 0.5005139112472534 0.0
14 268.12792979003945 cti 1 0.0
14 268.12792979003945 prohib 0.7644048929214478 0.0
14 268.12792979003945 vinculis 0.7086776494979858 0.0
14 268.12792979003945 appreciation 1 0.0
14 268.12792979003945 gratit

15 71.81399953437742 enum 0.558282196521759 0.0
15 71.81399953437742 flexibility 1 0.0
15 71.81399953437742 leeway 0.6320102214813232 0.0
15 71.81399953437742 freedom 0.5784385204315186 0.0
15 71.81399953437742 adaptability 1 0.0
15 71.81399953437742 endurance 0.40773600339889526 0.0
15 71.81399953437742 tremendously 0.3994724154472351 0.0
15 71.81399953437742 team 1 0.0
15 71.81399953437742 teams 0.682961106300354 0.0
15 71.81399953437742 crew 0.5394878387451172 0.0
15 71.81399953437742 player 1 0.0
15 71.81399953437742 players 0.6468743085861206 0.0
15 71.81399953437742 bot 0.5230127573013306 0.0
15 72.27645157385919 skill 1 0.46245203948177505
15 72.27645157385919 skills 0.7708162069320679 0.0
15 72.27645157385919 talent 0.6427983045578003 0.0
15 72.27645157385919 discipline 1 0.0
15 72.27645157385919 subfield 0.597419261932373 0.0
15 72.27645157385919 specialization 0.5085230469703674 0.0
15 75.32097401158262 provide 1 3.044522437723423
15 75.32097401158262 give 0.7738499641418457 

19 74.32732285996798 reconsider 0.5569480657577515 0.0
19 74.32732285996798 role 1 0.0
19 74.32732285996798 roles 0.6735905408859253 0.0
19 74.32732285996798 plaaay 0.4111264944076538 0.0
19 74.32732285996798 keywords 1 0.0
19 74.32732285996798 keyword 0.7382091879844666 0.0
19 74.32732285996798 titles 0.573992908000946 0.0
19 74.32732285996798 embed 1 0.0
19 74.32732285996798 inject 0.7326186895370483 0.0
19 74.32732285996798 incorporate 0.6480865478515625 0.0
19 74.32732285996798 salary 1 0.0
19 74.32732285996798 salaries 0.7508189678192139 0.0
19 74.32732285996798 stipend 0.7325254082679749 0.0
19 74.32732285996798 kk 1 0.0
19 74.32732285996798 ub 0.6419845223426819 0.0
19 74.32732285996798 gclid 0.6224895715713501 0.0
19 74.32732285996798 location 1 0.0
19 74.32732285996798 locations 0.6699954867362976 0.0
19 74.32732285996798 position 0.4978184998035431 0.0
19 74.32732285996798 fleet 1 0.0
19 74.32732285996798 army 0.5726014375686646 0.0
19 74.32732285996798 military 0.50874972343

20 66.70259590278161 preferrably 0.35120654106140137 0.0
20 66.70259590278161 unix 2 0.0
20 66.70259590278161 nix 2.8733590841293335 0.0
20 66.70259590278161 posix 1.392472267150879 0.0
20 66.70259590278161 linux 2.6197802424430847 0.0
20 66.70259590278161 ubuntu 2.074284553527832 0.0
20 66.70259590278161 windows 1 0.0
20 66.70259590278161 html 1 0.0
20 66.70259590278161 htm 0.591211199760437 0.0
20 66.70259590278161 xhtml 0.5737585425376892 0.0
20 66.70259590278161 php 1 0.0
20 66.70259590278161 lighttpd 0.540486752986908 0.0
20 66.70259590278161 srv 0.5323005318641663 0.0
20 66.70259590278161 candidate 1 0.0
20 66.70259590278161 candidates 0.6694498062133789 0.0
20 66.70259590278161 applicant 0.6307728290557861 0.0
20 66.70259590278161 background 1 0.0
20 66.70259590278161 foreground 0.5441020727157593 0.0
20 66.70259590278161 backgrounds 0.4340760409832001 0.0
20 66.70259590278161 advantage 1 0.0
20 66.70259590278161 advantages 0.6166207790374756 0.0
20 66.70259590278161 disadvantag

In [28]:
ranked_resumes = []
for i in range(num_resumes):
    ranked_resumes.append((score[i], i))    
ranked_resumes.sort(reverse = True)
for a, i in ranked_resumes:
    if list(resumes)[i] != '.DS_Store':
        print(list(resumes)[i], ':', a)

cv50 : 848.2244209343917
cv61 : 290.0897429600622
cv36 : 215.21116481910263
cv55 : 94.09680660970866
cv57 : 84.65091181927765
cv63 : 84.18687413803237
cv62 : 81.41001888702947
cv56 : 77.53724306621127
cv66 : 74.32732285996798
cv67 : 71.85267783385399
cv59 : 63.895706645456286
cv2 : 55.724128414942214
cv64 : 41.76674053098422
cv65 : 39.777480352150306
cv3 : 36.41633911877955
cv54 : 36.0104465624718
cv58 : 35.42511191790185
cv53 : 30.781095244701476
cv52 : 23.874102696449867
cv51 : 17.862940586667772
cv4 : 0
